# BrainAge Project

In [1]:
import pandas as pd
import numpy as np
import nibabel as nib
from sklearn.model_selection import KFold
from sklearn.svm import SVR
# from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_regression
import os
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error
from nilearn.input_data import NiftiMasker
import random
import matplotlib.pyplot as plt
import datetime

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected

In [2]:
# Parameters
toy = False
run_all = False
path = '/home/ubuntu/fmriprep/'
output_dir = '../output/'

n_jobs = 20 #amount of cores
cv=4
log_file = datetime.datetime.now().strftime("%y-%m-%d-%H-%M-%S")

In [3]:
# Load subject ids
if run_all:
    files = os.listdir(path)
    subj = [n[4:9] for n in files if n.startswith('sub')] #here we will filter the file names to leave only subj number as bellow

    input_dir = '/home/ubuntu/'
    file = 'subjlist_wfiles_CNP.txt'
    with open(input_dir+file, 'r') as f:
        lines = f.readlines()

    subj = [n.strip()[4:] for n in lines]

#     This is only done once
#     random_subj = subj[:]
#     random.shuffle(random_subj)

In [4]:
# if toy:
#     subj = subj[:8]
# random_subj


In [5]:
# Load ages
# df = pd.read_csv('participants.tsv', sep='\t')

# # take age from subj id
# y_age = []
# for i in random_subj:
#     a = df.loc[df['participant_id'] == 'sub-'+i] #whole row
#     y_age.append(int(a.age))


# # Load IQ
# df = pd.read_csv('add file here', sep='\t')

# # take age from subj id
# y_age = []
# for i in subj:
#     a = df.loc[df['participant_id'] == 'sub-'+i] #whole row
#     y_iq.append(int(a.id))


In [6]:
# # Load volumes for each subject into a dictionary
# d = {}
# for i in range(len(subj)):
#     img = nib.load(path+'sub-'+subj[i]+'_T1w_space_MNI152NLin2009cAsym_preproc.nii.gz').get_fdata()
#     d[subj[i]] = img

if run_all:
    filenames = []
    # data = []
    for i in range(len(random_subj)):
    #     img = nib.load(path+'sub-'+subj[i]+/anat/'sub-'+subj[i]+'_T1w_space_MNI152NLin2009cAsym_preproc.nii.gz').get_fdata()
    #     data.append(img)
        filenames.append(path+'sub-'+random_subj[i]+'/anat/sub-'+random_subj[i]+'_T1w_space-MNI152NLin2009cAsym_preproc.nii.gz')

# data  = np.array(data) 

## Let's see the size of a single volume
# data.shape

# # Flatten 3D volume and append to a list (or feed 3D volume to nilearn)
# X = []

# for i in d.values():
#     flattened = i.flatten()
#     X.append(flattened)
    
# # Turn list into array 
# X = np.array(X)

In [7]:
# filenames

In [8]:
# Affine resample

# from nilearn.image import resample_to_img
# # img = data.affine

# resampled_imgs = []
# resampled_imgs.append(nib.load(path+filenames[0]))
                      
# for i in range(1,len(filenames)):
#     resampled_img = resample_to_img(path+filenames[i], path+filenames[0])
#     resampled_imgs.append(resampled_img)

# for i in resampled_imgs:
#     print(i.affine)

In [9]:
if run_all:
    path_and_filenames = [path+n for n in filenames]

    nifti_masker = NiftiMasker(
        standardize=False,
        smoothing_fwhm=2, mask_strategy='epi',
        memory='nilearn_cache')  # cache options
    gm_maps_masked = nifti_masker.fit_transform(filenames)
    n_samples, n_features = gm_maps_masked.shape
    print("%d samples, %d features" % (n_samples, n_features))

In [10]:
if run_all:
    print("%d samples, %d features" % (n_samples, n_features))

In [11]:
if run_all:
    %matplotlib inline
    from nilearn import plotting
    plotting.plot_roi(nifti_masker.mask_img_)
    plotting.plot_roi(nifti_masker.mask_img_, bg_img=filenames[0])
    plt.savefig(output_dir+'nilearn_mask')

In [12]:
if run_all:
    X_train = gm_maps_masked[:84]
    X_test = gm_maps_masked[84:]
    y_train = y_age[:84]
    y_test = y_age[84:]
    random_subj_train = random_subj[:84]
    random_subj_test = random_subj[84:]

    np.savez_compressed(output_dir+log_file+'train_set',a=X_train, b=y_train, c=random_subj_train)
    np.savez_compressed(output_dir+log_file+'test_set',a=X_test, b=y_test, c=random_subj_test)
else:
    loaded = np.load(output_dir+'train_set.npz')
    X_train, y_train, random_subj_train = loaded['a'], loaded['b'], loaded['c']


In [13]:

# X_train, X_test, y_train, y_test = train_test_split(gm_maps_masked, y_age, test_size=0.31, random_state=42)

In [14]:
print(y_train.shape)
print(X_train.shape)

(84,)
(84, 1899247)


## Gridsearch 
### Models
* Linear Support vector regressor
    * C = 0.1, 1., 10.

### Feature selection
* ANOVA
    * 2000 features
    * 50 k
    * 100k features
    * 500k
    * Use all features (176*256*256=11534336)

In [15]:
from sklearn.pipeline import Pipeline

if toy:
    X_train = X_train[:8]
    y_train = y_train[:8]


# Remove features with too low between-subject variance
# variance_threshold = 

# Here we use a classical univariate feature selection based on F-test,
# namely Anova.
# feature_selection = SelectKBest(f_regression)

# We have our predictor (SVR), our feature selection (SelectKBest), and now,
# we can plug them together in a *pipeline* that performs the two operations
# successively:
               
               

anova_svr = Pipeline([
            ('variance_threshold', VarianceThreshold(threshold=.01)),
            ('anova', SelectKBest(f_regression)),
            ('svr', SVR())])

parameters = [{'anova__k': [1000, 2000, 3000, 4000, 5000],
               'svr__C': [0.00001, 0.0001, 0.001, 0.01]}]

# parameters = [{'anova__k': [2000],
#                'svr__C': [0.1,1]}]

grid = GridSearchCV(anova_svr, cv=cv, n_jobs=n_jobs, param_grid=parameters, scoring='neg_mean_absolute_error')
grid.fit(X_train, y_train)

# mean_scores = np.array(grid.cv_results_['neg_mean_absolute_error'])
# mean_scores = mean_scores.reshape(len(C_OPTIONS), -1, len(N_FEATURES_OPTIONS))
# # select score for best C
# mean_scores = mean_scores.max(axis=0)
# bar_offsets = (np.arange(len(N_FEATURES_OPTIONS)) *
#                (len(reducer_labels) + 1) + .5)






GridSearchCV(cv=4, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('variance_threshold', VarianceThreshold(threshold=0.01)), ('anova', SelectKBest(k=10, score_func=<function f_regression at 0x7f70b517c7b8>)), ('svr', SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False))]),
       fit_params=None, iid=True, n_jobs=20,
       param_grid=[{'anova__k': [5000, 100000, 'all'], 'svr__C': [1e-05, 0.0001, 0.001, 0.01]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_absolute_error', verbose=0)

In [16]:
print('best score: ',grid.best_score_)
print('best params: ',grid.best_params_)
print('\n\nfullresults: \n', grid.cv_results_)
df1 = pd.DataFrame(grid.cv_results_)
df1

best score:  -7.459315476190476
best params:  {'anova__k': 5000, 'svr__C': 0.01}


fullresults: 
 {'std_test_score': array([1.06220085, 1.06219944, 1.06218532, 1.06220306, 1.06220085,
       1.06219944, 1.06218532, 1.06220306, 1.06220085, 1.06219944,
       1.06218532, 1.06220306]), 'split1_train_score': array([-6.89364151, -6.89355794, -6.89272222, -6.88424603, -6.89364151,
       -6.89355794, -6.89272222, -6.88424603, -6.89364151, -6.89355794,
       -6.89272222, -6.88424603]), 'params': [{'anova__k': 5000, 'svr__C': 1e-05}, {'anova__k': 5000, 'svr__C': 0.0001}, {'anova__k': 5000, 'svr__C': 0.001}, {'anova__k': 5000, 'svr__C': 0.01}, {'anova__k': 100000, 'svr__C': 1e-05}, {'anova__k': 100000, 'svr__C': 0.0001}, {'anova__k': 100000, 'svr__C': 0.001}, {'anova__k': 100000, 'svr__C': 0.01}, {'anova__k': 'all', 'svr__C': 1e-05}, {'anova__k': 'all', 'svr__C': 0.0001}, {'anova__k': 'all', 'svr__C': 0.001}, {'anova__k': 'all', 'svr__C': 0.01}], 'split3_test_score': array([-6.66190452, -6.661

/home/ubuntu/.local/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/ubuntu/.local/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/ubuntu/.local/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/ubuntu/.local/lib/python3.5/site-pac

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_anova__k,param_svr__C,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,split3_test_score,split3_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,7.715599,0.839884,-7.459524,-7.276975,5000,1e-05,"{'anova__k': 5000, 'svr__C': 1e-05}",10,-6.185714,-7.684117,-8.728570,-6.893642,-8.261906,-7.004753,-6.661905,-7.525387,0.275484,0.130938,1.062201,0.334859
1,8.899364,0.782186,-7.459521,-7.276889,5000,0.0001,"{'anova__k': 5000, 'svr__C': 0.0001}",7,-6.185712,-7.684029,-8.728555,-6.893558,-8.261917,-7.004669,-6.661902,-7.525299,0.254899,0.112782,1.062199,0.334857
2,9.274367,0.902791,-7.459500,-7.276032,5000,0.001,"{'anova__k': 5000, 'svr__C': 0.001}",4,-6.185690,-7.683151,-8.728405,-6.892722,-8.262024,-7.003833,-6.661881,-7.524421,0.472087,0.157368,1.062185,0.334836
3,8.934201,0.854356,-7.459315,-7.267411,5000,0.01,"{'anova__k': 5000, 'svr__C': 0.01}",1,-6.185476,-7.674365,-8.727738,-6.884246,-8.262500,-6.995357,-6.661548,-7.515675,0.220187,0.170460,1.062203,0.334692
4,9.860977,1.106586,-7.459524,-7.276975,100000,1e-05,"{'anova__k': 100000, 'svr__C': 1e-05}",10,-6.185714,-7.684117,-8.728570,-6.893642,-8.261906,-7.004753,-6.661905,-7.525387,0.411454,0.196944,1.062201,0.334859
5,10.956813,1.285727,-7.459521,-7.276889,100000,0.0001,"{'anova__k': 100000, 'svr__C': 0.0001}",7,-6.185712,-7.684029,-8.728555,-6.893558,-8.261917,-7.004669,-6.661902,-7.525299,0.377143,0.238121,1.062199,0.334857
6,11.079412,1.387707,-7.459500,-7.276032,100000,0.001,"{'anova__k': 100000, 'svr__C': 0.001}",4,-6.185690,-7.683151,-8.728405,-6.892722,-8.262024,-7.003833,-6.661881,-7.524421,0.466082,0.273825,1.062185,0.334836
7,10.990370,1.462623,-7.459315,-7.267411,100000,0.01,"{'anova__k': 100000, 'svr__C': 0.01}",1,-6.185476,-7.674365,-8.727738,-6.884246,-8.262500,-6.995357,-6.661548,-7.515675,0.197961,0.161812,1.062203,0.334692
8,51.205935,15.975753,-7.459524,-7.276975,all,1e-05,"{'anova__k': 'all', 'svr__C': 1e-05}",10,-6.185714,-7.684117,-8.728570,-6.893642,-8.261906,-7.004753,-6.661905,-7.525387,3.596184,0.242564,1.062201,0.334859
9,58.925374,16.365395,-7.459521,-7.276889,all,0.0001,"{'anova__k': 'all', 'svr__C': 0.0001}",7,-6.185712,-7.684029,-8.728555,-6.893558,-8.261917,-7.004669,-6.661902,-7.525299,2.384770,0.246853,1.062199,0.334857


In [17]:
age_pred = grid.predict(X_train)
mean_abs_error = np.mean(np.abs(age_pred-y_train))
df1 = pd.DataFrame(grid.cv_results_)

with open(output_dir+'log_'+log_file+'.txt', '+a') as f:
    f.write('best score: '+str(grid.best_score_))
    f.write('best params: '+str(grid.best_params_))
    f.write('\n\nfullresults: \n'+str(grid.cv_results_)+'\n')
    f.write(str(df1)+'\n')
    f.write(str(age_pred)+'\n')
    f.write(str(y_train)+'\n')
    f.write(str(mean_abs_error)+'\n')
    

/home/ubuntu/.local/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/ubuntu/.local/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/ubuntu/.local/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/ubuntu/.local/lib/python3.5/site-pac

In [18]:
# age_pred = grid.predict(gm_maps_masked)
# np.mean(np.abs(age_pred-y_age))
# y_age

In [19]:
# r2 score